In [1]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
  color: black !important;
}

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [4]:
df = pd.read_csv('hc_ds_glassdoor_jobs.csv')
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,...,Same_State,Comp_Age,job_title,Seniority,job_descript_lc,python,sql,aws,scala,pyspark
0,Data Scientist,108145,"Secure our Nation, Ignite your Future\n\nSumma...",4.1,ManTech\n4.1,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968,Company - Public,...,0,52,data scientist,na,3490,0,1,0,0,0
1,Data Scientist,108145,Who We Are\n\nWere Americas largest mortgage l...,3.9,Quicken Loans\n3.9,"Detroit, MI","Detroit, MI",10000+ employees,1985,Company - Private,...,1,35,data scientist,na,4376,1,1,0,0,0
2,Data Scientist,108145,GEICO’s Data Science team uses predictive anal...,3.3,GEICO\n3.3,"Chevy Chase, MD","Washington, DC",10000+ employees,1936,Subsidiary or Business Segment,...,0,84,data scientist,na,3785,1,1,1,0,0
3,Data Scientist,108145,*Organization and Job ID**\nJob ID: 310781\n\n...,3.8,Pacific Northwest National Laboratory\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,...,1,55,data scientist,na,5144,0,0,0,0,0
4,Data Scientist,108145,JOB CATEGORY:\n\nInformation Services\n\nREQUI...,3.9,Mars\n3.9,New York State,"Mc Lean, VA",10000+ employees,1911,Company - Private,...,0,109,data scientist,na,5423,1,0,1,0,1


In [5]:
df.columns

Index(['Job Title', 'Salary Estimate', 'Job Description', 'Rating',
       'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors',
       'min_salary', 'max_salary', 'avg_salary', 'CompName', 'State',
       'Same_State', 'Comp_Age', 'job_title', 'Seniority', 'job_descript_lc',
       'python', 'sql', 'aws', 'scala', 'pyspark'],
      dtype='object')

In [7]:
df_model = df[['avg_salary','Rating','Size','Type of ownership', 'Sector','Revenue','State','Comp_Age','job_title','Seniority','Same_State','job_descript_lc','python','sql','aws','scala','pyspark']]

In [8]:
df_dum = pd.get_dummies(df_model, drop_first = True)

In [9]:
df_dum.head()

,avg_salary,Rating,Comp_Age,Same_State,job_descript_lc,python,sql,aws,scala,pyspark,...,State_WV,State_te,job_title_data engineer,job_title_data scientist,job_title_director,job_title_manager,job_title_mle,job_title_na,Seniority_na,Seniority_sr
0,126.5,4.1,52,0,3490,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,126.5,3.9,35,1,4376,1,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,126.5,3.3,84,0,3785,1,1,1,0,0,...,0,0,0,1,0,0,0,0,1,0
3,126.5,3.8,55,1,5144,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
4,126.5,3.9,109,0,5423,1,0,1,0,1,...,0,0,0,1,0,0,0,0,1,0


###### Train Test Split

In [11]:
from sklearn.model_selection import train_test_split

X = df_dum.drop('avg_salary', axis = 1)
y = df_dum.avg_salary.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state =42)

###### Multiple Linear Regression

In [12]:
import statsmodels.api as sm

X_sm = X = sm.add_constant(X)
model = sm.OLS(y, X_sm)
model.fit().summary()

/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.364
Model:                            OLS   Adj. R-squared:                  0.291
Method:                 Least Squares   F-statistic:                     4.975
Date:                Wed, 20 May 2020   Prob (F-statistic):           3.63e-45
Time:                        13:32:22   Log-Likelihood:                -5165.3
No. Observations:                1105   AIC:                         1.056e+04
Df Residuals:                     990   BIC:                         1.114e+04
Df Model:                         114                                         
Covariance Type:            nonrobust                                         
====================================================================================================================
                                                       coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
const                                               57.6520     28.342      2.034      0.042       2.034     113.270
Rating                                              -0.8441      1.205     -0.700      0.484      -3.209       1.521
Comp_Age                                             0.0185      0.028      0.669      0.503      -0.036       0.073
Same_State                                          -0.7223      1.965     -0.368      0.713      -4.578       3.133
job_descript_lc                                      0.0005      0.001      0.762      0.446      -0.001       0.002
python                                               3.5808      2.139      1.674      0.094      -0.617       7.779
sql                                                 -2.1153      2.056     -1.029      0.304      -6.149       1.919
aws                                                 -1.0628      2.198     -0.484      0.629      -5.376       3.250
scala                                                4.2407      2.299      1.845      0.065      -0.271       8.752
pyspark                                            -11.5622      5.643     -2.049      0.041     -22.636      -0.489
Size_1 to 50 employees                               0.2303      5.236      0.044      0.965     -10.044      10.504
Size_10000+ employees                               10.3763      5.419      1.915      0.056      -0.257      21.009
Size_1001 to 5000 employees                          7.7465      4.178      1.854      0.064      -0.452      15.945
Size_201 to 500 employees                            2.6848      4.499      0.597      0.551      -6.145      11.514
Size_5001 to 10000 employees                         7.4947      5.242      1.430      0.153      -2.793      17.782
Size_501 to 1000 employees                           1.1792      4.581      0.257      0.797      -7.811      10.170
Size_51 to 200 employees                             6.0991      4.766      1.280      0.201      -3.254      15.452
Size_Unknown                                        14.4549     10.662      1.356      0.176      -6.468      35.378
Type of ownership_College / University               0.3507     12.736      0.028      0.978     -24.642      25.343
Type of ownership_Company - Private                  4.1936      4.525      0.927      0.354      -4.686      13.073
Type of ownership_Company - Public                   4.1336      4.832      0.856      0.392      -5.348      13.615
Type of ownership_Contract                          14.4906     19.129      0.758      0.449     -23.048      52.029
Type of ownership_Government                        10.1784      9.371      1.086      0.278      -8.211      28.568
Type of ownership_Hospital               

In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

lm = LinearRegression()
lm.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [14]:
np.mean(cross_val_score(lm,X_train,y_train,scoring = 'neg_mean_absolute_error', cv=3))

-23.116990991648425

###### Random Forest Model

In [16]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor()
np.mean(cross_val_score(rf, X_train, y_train, scoring = 'neg_mean_absolute_error', cv=3))

/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


-23.163434047280465

###### GridSearchCV

In [17]:
from sklearn.model_selection import GridSearchCV

parameters = {'n_estimators' :range(10,300,10), 'criterion':('mse','mae'), 'max_features':('auto','sqrt','log2')}
gs = GridSearchCV(rf, parameters, scoring = 'neg_mean_absolute_error', cv=3)
gs.fit(X_train, y_train)

GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'criterion': ('mse', 'mae'),
  